# 1. Import libraries

In [38]:
import csv 
import pandas as pd
import ast
import re
import glob
import os

# 2. Declaring vars and functions

In [39]:
def string_to_dict(string):
    if isinstance(string, dict):
        return string
    try:
        return ast.literal_eval(string)
    except:
        return ast.literal_eval(string.replace('"','\\"'))
        
def record_nodes(user):
    if user[0] not in nodes:
        nodes[user[0]] = user
    elif (user[3] > nodes[user[0]][3]):
        nodes[user[0]] = user
    return user[0]
          
def extract_source(tweet, quoted=False):   
    try:
        if quoted:
            u = tweet['user']
            return record_nodes([u['username'].lower(), u['followersCount'], u['friendsCount'], str(tweet['date'])[:10]])
        else:
            u = ast.literal_eval(tweet['Author'].values[0])
            return record_nodes([u['username'].lower(), u['followersCount'], u['friendsCount'], str(tweet['date'])[:10]])

    except Exception as e:
        display(tweet)
        raise e
        

def extract_mention_targets(tweet):   
    text = tweet['content'] if isinstance(tweet['content'], str) else tweet['content'].values[0] 
    users = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)', text, flags=re.IGNORECASE)
    return [record_nodes([u.lower(), 0, 0,str(tweet['date'])[:10]]) for u in users]

def extract_mention_edges(tweet, quoted=False):
    # source, Target, Label, Weigth, Tweet Id 
    source = extract_source(tweet, quoted)
    targets = extract_mention_targets(tweet)
    for t in targets:
        edges.append([source, t, 'MENTION', 1, tweet['id'].values[0] if not quoted else tweet['id']])

    return source

def extract_quoted_target(q):
    return record_nodes([q['user']['username'].lower(), q['user']['followersCount'], q['user']['friendsCount'], str(q['date'])[:10]])

def extract_quoted_edge(quoted_tweet, source):
    # source, Target, Label, Weigth, Tweet ID
    q = string_to_dict(quoted_tweet)
    u = extract_quoted_target(q)
    edges.append([source, u, 'QUOTE', 1, q['id']])
    
def process_tweet(tweet, quoted = False):
    source = extract_mention_edges(tweet, quoted)
    
    if not quoted and pd.notnull(tweet['quotedTweet']).all():  
        quotedTweet = tweet['quotedTweet'].values[0] 
        extract_quoted_edge(quotedTweet, source) # quoted edge
        process_tweet(string_to_dict(quotedTweet), True) #process the tweet
        
    elif quoted and tweet['quotedTweet']:
        quotedTweet = tweet['quotedTweet']
        extract_quoted_edge(quotedTweet, source) 
        process_tweet(quotedTweet, True)

def process_file(filename):
    print(filename)
    errors = 0 
    df = pd.read_excel(filename, index_col=0).reset_index()
    display(df.head())
    for index, row in df.iterrows(): #process tweets
        try: 
            process_tweet(df.iloc[[index]])
        except Exception as e:   
            print("Index: ", index)
            print(type(df.iloc[[index]]))
            raise e
            errors += 1

    df_nodes = pd.DataFrame.from_dict(nodes, orient='index', columns=['Label', 'Followers', 'Friends', 'Date']) 
    df_nodes = df_nodes[['Label', 'Followers', 'Friends']]
    df_nodes.to_csv("data/nets/nodes_" + filename.replace('.xlsx', '.csv').replace('data/raw_data\\', ''), index=True)
    
    df_edges = pd.DataFrame(edges, columns=['Source', 'Target', 'Label', 'Weight', 'Tweet ID']) 
    df_edges = df_edges.drop_duplicates(subset=['Source', 'Target', 'Label', 'Tweet ID'], keep='last')
    df_edges = df_edges[['Source', 'Target', 'Label', 'Weight']]
    df_edges = df_edges.groupby(['Source', 'Target', 'Label']).agg({'Weight': 'sum'}).reset_index()
    df_edges.to_csv("data/nets/edges_" + filename.replace('.xlsx', '.csv').replace('data/raw_data\\', ''), index=False)
    print (errors)
    return [filename, len(df), len(df_nodes), len(df_edges), errors]

# 3. Process files

In [40]:
import json
pd.set_option('display.max_colwidth', None)
stats = []
PATH = 'data/raw_data/'

for filename in glob.glob(os.path.join(PATH, '*.xlsx')):
    nodes = {}
    edges = [] 
    stats.append(process_file(filename))
    
df = pd.DataFrame(stats, columns = ['Filename', 'No. Tweets', 'No. Nodes', 'No. Edges', 'No. Errors'])
df.to_csv('data\global_net_stats.csv', index = False)
len(df)

data/raw_data\brasilEvento3.xlsx


,date,content,id,outlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,retweetedTweet,quotedTweet,Author,username,followers_list,friends_list
0,2020-05-26,O Brasil já sabe que a família Bolsonaro interfere na Polícia Federal para se blindar das investigações. Não podemos deixar que o Caso Marielle seja federalizado nesse momento. #FederalizaçãoNão https://t.co/0wd0fyWtGv,1265432499263503872,['http://www.federalizacaonao.org'],0,0,0,0,1265432499263503872,pt,NaN,NaN,"{'username': 'jeivison_jose', 'displayname': 'Jeivison José', 'id': 1199717598482579456, 'description': 'Fotógrafo por acaso.', 'rawDescription': 'Fotógrafo por acaso.', 'descriptionUrls': [], 'verified': False, 'created': '2019-11-27T15:52:49+00:00', 'followersCount': 5, 'friendsCount': 99, 'statusesCount': 62, 'favouritesCount': 380, 'listedCount': 0, 'mediaCount': 1, 'location': 'São Paulo', 'protected': False, 'linkUrl': 'https://jeivison.myportfolio.com/', 'linkTcourl': 'https://t.co/YuG03T6vCp', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1265431002631081984/-HsQy-pH_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1199717598482579456/1590537237', 'url': 'https://twitter.com/jeivison_jose'}",jeivison_jose,5,99
1,2020-05-26,Só até amanhã. \nO Brasil já sabe que a família Bolsonaro interfere na Polícia Federal para se blindar das investigações. Não podemos deixar que o Caso Marielle seja federalizado nesse momento. #FederalizaçãoNão https://t.co/m0SRsmqpsc,1265432263048790016,['http://www.federalizacaonao.org'],0,0,0,0,1265432263048790016,pt,NaN,NaN,"{'username': 'peraindameufi', 'displayname': 'Cacto Tchóp da Juliette🌵', 'id': 1186128416, 'description': 'Pernambucana, Recifense Vivendo no Cerrado 🏜\n⚡️ Amor ao Trance Rave Festival ♪ Autismo🧩 #FeministaTodoDia Leonina ⚠️INSTA️❣ Instagram.com/peraiiinda/', 'rawDescription': 'Pernambucana, Recifense Vivendo no Cerrado 🏜\n⚡️ Amor ao Trance Rave Festival ♪ Autismo🧩 #FeministaTodoDia Leonina ⚠️INSTA️❣ https://t.co/eTOTbXks18', 'descriptionUrls': [{'text': 'Instagram.com/peraiiinda/', 'url': 'http://Instagram.com/peraiiinda/', 'tcourl': 'https://t.co/eTOTbXks18', 'indices': [126, 149]}], 'verified': False, 'created': '2013-02-16T12:44:48+00:00', 'followersCount': 2588, 'friendsCount': 2315, 'statusesCount': 7957, 'favouritesCount': 26374, 'listedCount': 8, 'mediaCount': 440, 'location': 'Brasília DF', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1322196825407950848/mWv8egPS_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1186128416/1604071518', 'url': 'https://twitter.com/peraindameufi'}",peraindameufi,2588,2315
2,2020-05-26,"Disseram:""Bolsonaro é a segunda maior vergonha do Brasil""... Por isso nossa empresa investigada pela Polícia Federal assinou contrato de MILHÕES DE REAIS com Sergio Moro...? 🤨🤨🤨\n\nEntenderam?\n\nVia @domlancellotti\n\n#MoroNaCadeia 💥 https://t.co/BWvDXHSd9P",1265432001437470976,[],2,6,16,0,1265431931648445952,pt,NaN,NaN,"{'username': 'a_jessicao', 'displayname': 'Jessicão, a Opressora', 'id': 1248830800281378817, 'description': '🗣️| Vereadora na cidade de Londrina-PR\n📢 | Presidente do Direita Paraná Oficial\n🇧🇷 | #bolsonaro2022\n📱 | 43 9.96484741', 'rawDescription': '🗣️| Vereadora na cidade de Londrina-PR\n📢 | Presidente do Direita Paraná Oficial\n🇧🇷 | #bolsonaro2022\n📱 | 43 9.96484741', 'descriptionUrls': [], 'verified': False, 'created': '2020-04-11T04:31:02+00:00', 'followersCount': 22267, 'friendsCount': 325, 'statusesCount': 1890, 'favouritesCount': 6316, 'listedCount': 11, 'mediaCount': 696, 'location': 'Londrina, Brasil', 'protected': False, 'linkUrl': 'https://www.facebook.com/jessicaoaopressora', 'linkTcourl': 'https://t.co/OcZryakVRU', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1248844060791656448/BiJQvViI_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1248830800281378817/1613063982', 'url': 'https

0
data/raw_data\colombiaEvento1.xlsx


,index,date,content,id,outlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,retweetedTweet,quotedTweet,Author,username,followers_list,friends_list
0,171258,2020-09-10,"@RevistaSemana NO MAS BALAS !!\nquien dió la orden !!!\n\nEn Fontibon hoy 10-Septiembre-2020, @PoliciaColombia abren fuego contra manifestantes https://t.co/UOs1CtE9nN",1304208027281690880,[],0,0,0,0,1304201250326614016,es,NaN,NaN,"{'username': '_COLOMBIAHUMANA', 'displayname': 'LA VERDAD NOS HARÁ LIBRES!👨\u200d🌾👨\u200d🎓🥑🌾🌲🚯🚮🇨🇴', 'id': 1282323464280584192, 'description': 'El debate del Gobierno entre derecha e izquierda no es el debate de esta generación! El debate es entre progreso y atraso... Nosotros somos progresismo 👊🏾', 'rawDescription': 'El debate del Gobierno entre derecha e izquierda no es el debate de esta generación! El debate es entre progreso y atraso... Nosotros somos progresismo 👊🏾', 'descriptionUrls': [], 'verified': False, 'created': '2020-07-12T14:38:41+00:00', 'followersCount': 1487, 'friendsCount': 590, 'statusesCount': 33525, 'favouritesCount': 15826, 'listedCount': 2, 'mediaCount': 9073, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1294688165542928390/kskxQIpO_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1282323464280584192/1613740078', 'url': 'https://twitter.com/_COLOMBIAHUMANA'}",_COLOMBIAHUMANA,1487,590
1,171259,2020-09-10,"@PoliciaColombia Institución con poca credibilidad y preparo, agentes ordinarios que a veces se confunden con gamines por su forma coloquial de hablar, sin valores y nada cultos, perdieron mi respeto.",1304208025750774016,[],0,0,0,0,1304183024217711104,es,NaN,NaN,"{'username': 'Cuest9', 'displayname': 'Cuest', 'id': 1240518196970618881, 'description': '#Bitcoin\n#nosugar', 'rawDescription': '#Bitcoin\n#nosugar', 'descriptionUrls': [], 'verified': False, 'created': '2020-03-19T05:59:32+00:00', 'followersCount': 10, 'friendsCount': 103, 'statusesCount': 197, 'favouritesCount': 846, 'listedCount': 0, 'mediaCount': 0, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1304207463890259970/7NN8oQda_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1240518196970618881/1599782341', 'url': 'https://twitter.com/Cuest9'}",Cuest9,10,103
2,171260,2020-09-10,"@somosraya @PoliciaBogota @ProteccionPonal @Policia924FM @TransitoBta @PoliciaColombia @AnimalesBOG Mañana les ordenan que lleven a los hijos a las protestas, y éstos obedecen. #QuienDioLaOrden ?",1304208007354609920,[],1,0,0,0,1304150393438573056,es,NaN,NaN,"{'username': 'Cyanider_', 'displayname': '𝓒𝒚ᴀ𝒏ɪ𝒅eЯ', 'id': 77595198, 'description': 'All will bear a part of me, let this\nhollow shell set your free to live as one...', 'rawDescription': 'All will bear a part of me, let this\nhollow shell set your free to live as one...', 'descriptionUrls': [], 'verified': False, 'created': '2009-09-26T22:50:37+00:00', 'followersCount': 3294, 'friendsCount': 3352, 'statusesCount': 7038, 'favouritesCount': 10321, 'listedCount': 0, 'mediaCount': 662, 'location': 'Bucaramsterdam', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1377301149850669062/0XEA_umO_normal.png', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/77595198/1614792236', 'url': 'https://twitter.com/Cyanider_'}",Cyanider_,3294,3352
3,171261,2020-09-10,@elcinicoreyrana @PoliciaColombia @verdolaga2323 Estamos hablando con base a los hechos que dieron lugar al levantamiento del pueblo.\n\nY seguimos siendo victimas de sus atropellos.,1304208005936886016,[],1,0,0,0,1304183024217711104,es,NaN,NaN,"{'username': 'SargentoYosa', 'displayname': '🇨🇴Sargento (r) Yosa 🇨🇴', 'id': 1409414106, 'description': 'Exmilitar, Abogado penalista, Quiero una Colombia justa y equitativa, celebro La Paz.\n#ReservaSocialporlaPaz \n#NegativoParaU

0
data/raw_data\colombiaEvento2.xlsx


...

,index,date,content,id,outlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,retweetedTweet,quotedTweet,Author,username,followers_list,friends_list
0,16401,2020-11-16,"Ante la crisis política en #Perú, la Federación blanquiroja confirmó que hay las garantías por parte de la policía del país para el partido que se jugará a las 19:30 en el Monumental del Lima ante #Argentina \n\n#EliminatoriasQatar2022\n#TeamDeportivoEc https://t.co/YWPrSMe5PY",1328483150784238080,[],0,0,0,0,1328483150784238080,es,NaN,NaN,"{'username': 'TeamDeportivoEC', 'displayname': 'TeamDeportivo', 'id': 1225611173476868097, 'description': 'Página especializada en el fútbol de Ecuador y del mundo \n#TeamDeportivoEc', 'rawDescription': 'Página especializada en el fútbol de Ecuador y del mundo \n#TeamDeportivoEc', 'descriptionUrls': [], 'verified': False, 'created': '2020-02-07T02:44:34+00:00', 'followersCount': 20, 'friendsCount': 242, 'statusesCount': 3604, 'favouritesCount': 63, 'listedCount': 0, 'mediaCount': 1673, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1347739829572939777/2hwz3VZz_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1225611173476868097/1581043882', 'url': 'https://twitter.com/TeamDeportivoEC'}",TeamDeportivoEC,20,242
1,16402,2020-11-16,"📍 La Policía llegó hasta su vivienda del hombre, quien confirmó que no había viajado a Lima para protestar ☝️https://t.co/CNrJzCKigp",1328481295534596096,['https://radiokaribena.pe/hombre-aparece-en-lista-de-desaparecidos-pero-vive-en-arequipa/'],0,0,0,0,1328481295534596096,es,NaN,NaN,"{'username': 'karibenape', 'displayname': 'La Karibeña', 'id': 1683606829, 'description': 'Twitter oficial ¡Espectáculos, cumbia y más! Escúchanos en los 94.9FM', 'rawDescription': 'Twitter oficial ¡Espectáculos, cumbia y más! Escúchanos en los 94.9FM', 'descriptionUrls': [], 'verified': False, 'created': '2013-08-19T15:51:18+00:00', 'followersCount': 25800, 'friendsCount': 241, 'statusesCount': 56534, 'favouritesCount': 800, 'listedCount': 60, 'mediaCount': 34217, 'location': 'Lima - Perú', 'protected': False, 'linkUrl': 'http://www.radiokaribena.pe/', 'linkTcourl': 'http://t.co/AlJjDMaF2q', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1343574315632111621/4sskp2E9_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1683606829/1609768842', 'url': 'https://twitter.com/karibenape'}",karibenape,25800,241
2,16403,2020-11-16,"@CARLOSFMEJIA Imaginamos que el ""exitoso"" grupo de Lima y el mejor ejército y policía de Latinoamérica ya tienen controlado todo XD https://t.co/YzjqlFLpZX",1328475683257987072,[],0,0,1,0,1328364341519938048,es,NaN,NaN,"{'username': 'EdFor20', 'displayname': 'nerver walk alone', 'id': 1267821752370860033, 'description': '', 'rawDescription': '', 'descriptionUrls': [], 'verified': False, 'created': '2020-06-02T14:14:06+00:00', 'followersCount': 416, 'friendsCount': 794, 'statusesCount': 2960, 'favouritesCount': 25721, 'listedCount': 0, 'mediaCount': 72, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1330815250220933121/AvtJGWFw_normal.jpg', 'profileBannerUrl': None, 'url': 'https://twitter.com/EdFor20'}",EdFor20,416,794
3,16404,2020-11-16,"🇵🇪#Peru #MarchaNacional\n#Lima \n¡¡¡QUE COBARDE ES LA POLICÍA NACIONAL !!!\nBlanca no tienen ni la conciencia ni el alma, por eso el Pueblo los quiere matar. DIOS LOS MALDICE‼️ \nhttps://t.co/2fzKw1uSb9 \n\n#JackBryanPintadoSanchez \n#HeroeDelBicentenario \n#MartirDeLaDemocracia https://t.co/BwWDjsZpfG",1328471482037637120,['https://youtu.be/6hKTw_5oyp8'],0,1,1,0,1328471482037637120,es,NaN,NaN,"{'username': 'ContraApostasia', 'displayname': 'ContraApostasia', 'id': 1215368976240447488, 'description': '© ✝️ The Militia of the Soldiers of Jesus Christ®™ ⚔️ (MSIC©®™) 👼', 'rawDescription': '© ✝️ The Militia of the Soldiers of Jesus Chr

0
data/raw_data\peruEvento3.xlsx


,date,content,id,outlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,retweetedTweet,quotedTweet,Author,username,followers_list,friends_list
0,2020-11-13,"En sullana-piura, tumbes y Arequipa la policía esta marchando con el pueblo de manera pacífica y está brindando mascarillas a los que no tienen, son un gran ejemplo para los tombos inútiles de Lima. \n#PeruVsChile",1327400807550116096,[],2,7,36,0,1327400807550116096,es,NaN,NaN,"{'username': 'godispjm_', 'displayname': 'Angel♀️', 'id': 1001652185061560320, 'description': 'no words to describe jimin. he is a angel.', 'rawDescription': 'no words to describe jimin. he is a angel.', 'descriptionUrls': [], 'verified': False, 'created': '2018-05-30T02:31:05+00:00', 'followersCount': 2326, 'friendsCount': 725, 'statusesCount': 66750, 'favouritesCount': 84691, 'listedCount': 23, 'mediaCount': 10282, 'location': ""jimin it's Korean boy"", 'protected': False, 'linkUrl': 'https://pjmsupport.carrd.co/', 'linkTcourl': 'https://t.co/zchsDQ1zR9', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1376285826187984897/EU9uur5D_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1001652185061560320/1617334371', 'url': 'https://twitter.com/godispjm_'}",godispjm_,2326,725
1,2020-11-13,"@AdrianzenEduard La gente miraflorina, los de la Molina, San Isidro, San Borja ,etc. Debería ir al centro de Lima. A ver si tocan a uno y los padres hacen mierda a la policía, esperemos que no les hagan nada #perúvschile #peruchile",1327400073064549888,[],0,1,14,0,1327395767489794048,es,NaN,NaN,"{'username': 'geryruiz08', 'displayname': 'Geraldine⚓', 'id': 159964181, 'description': '', 'rawDescription': '', 'descriptionUrls': [], 'verified': False, 'created': '2010-06-26T20:19:30+00:00', 'followersCount': 34, 'friendsCount': 122, 'statusesCount': 416, 'favouritesCount': 430, 'listedCount': 0, 'mediaCount': 45, 'location': 'Perú', 'protected': False, 'linkUrl': 'https://www.instagram.com/geryruiz08/', 'linkTcourl': 'https://t.co/onnnjXqgfN', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/988141856834965504/9QB7bp91_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/159964181/1523233254', 'url': 'https://twitter.com/geryruiz08'}",geryruiz08,34,122
2,2020-11-13,"@Yapa_qu399 @CharitoABC @PoliciaPeru @MininterPeru @pcmperu @MindefPeru @LucianoRevoredo @abeja_peru No, en provincia, todo tranquilo y la policía no fue violenta. Los manifestantes se fueron a casa temprano y ya. Solo en Lima hicieron vandalismo. Vean cómo quedaron las calles llenas de piedras, vidrios rotos, cajeros vandalizados, monumentos pintarrajeados. Delincuentes!",1327398865268568064,[],0,1,0,0,1327258852652442112,es,NaN,NaN,"{'username': 'Yolimalintzin', 'displayname': 'Yolimalintzin', 'id': 2249728525, 'description': '#TraditionalCatholic #Pachamama is idolatry #ProLife #VOX_es #MAGA #Patriots #Communism=misery & chaos. Stop #GenderIdeology', 'rawDescription': '#TraditionalCatholic #Pachamama is idolatry #ProLife #VOX_es #MAGA #Patriots #Communism=misery & chaos. Stop #GenderIdeology', 'descriptionUrls': [], 'verified': False, 'created': '2013-12-17T03:21:15+00:00', 'followersCount': 1818, 'friendsCount': 2021, 'statusesCount': 27647, 'favouritesCount': 43692, 'listedCount': 2, 'mediaCount': 1035, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1368570331921793025/OIjUynOC_normal.jpg', 'profileBannerUrl': None, 'url': 'https://twitter.com/Yolimalintzin'}",Yolimalintzin,1818,2021
3,2020-11-13,Explíqueme porque la policía camina a su lado y en el centro de lima nos cogen a pedrones y gases https://t.co/ZMJwIns82R,1327396636809712128,['https://twitter.com/postmoderngrrl/status/1327364631229321221'],0,0,0,0,1327396636809712128,es,NaN,"{'url': 'https://twitter.com/postmoderngrrl/status/1327364631229321221', 'date': '2020-11-13T21:36:03+00:00', 'content': 'Manifestantes en LA MOLINA marchando ahor

0


8

In [41]:
stats

[['data/raw_data\\brasilEvento3.xlsx', 118026, 30896, 77035, 0],
 ['data/raw_data\\colombiaEvento1.xlsx', 241812, 74388, 299236, 0],
 ['data/raw_data\\colombiaEvento2.xlsx', 406395, 68444, 293559, 0],
 ['data/raw_data\\colombiaEvento3.xlsx', 444270, 58768, 199960, 0],
 ['data/raw_data\\ecuadorEvento1.xlsx', 156768, 34702, 178861, 0],
 ['data/raw_data\\ecuadorEvento2.xlsx', 204141, 36157, 207256, 0],
 ['data/raw_data\\peruEvento1.xlsx', 98730, 44464, 249682, 0],
 ['data/raw_data\\peruEvento3.xlsx', 138416, 27061, 173650, 0]]